In [73]:
'''
Задача 1
В этом задании вам снова понадобится ваша виртуальная машина.
Напишите программу, которая подключается к вашей виртуальной машине по SSH и читает содержимое файла /etc/services. 
Далее, используя регулярные выражения, в данных, полученных из этого файла,
найдите все сетевые сервисы, использующие четырехзначные UDP-порты, начинающиеся с 5-ки. 
Выведите список этих сервисов.
'''
import sys
import paramiko
import re


password = #тут пароль

host = '192.168.56.101'
user = 'wiz'
port = 22

try:
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname=host, username=user, password=password, port=port)
    stdin, stdout, stderr = client.exec_command('cat /etc/services')
    data = stdout.read().decode('utf-8') + stderr.read().decode('utf-8')
    print(re.findall('\w.*\D5\d{3}.*udp', data))

    
finally:
    client.close()

['rfe\t\t5002/udp', 'mmcc\t\t5050/udp', 'sip\t\t5060/udp', 'sip-tls\t\t5061/udp', 'aol\t\t5190/udp', 'xmpp-client\t5222/udp', 'xmpp-server\t5269/udp', 'cfengine\t5308/udp', 'mdns\t\t5353/udp', 'postgresql\t5432/udp', 'freeciv\t\t5556/udp', 'amqp\t\t5672/udp', 'ggz\t\t5688/udp', 'noclog\t\t5354/udp', 'hostmon\t\t5355/udp', 'rplay\t\t5555/udp']


In [86]:
'''
Задача 2
Напишите программу, которая подключается к вашей виртуальной машине по SSH и 
получает список всех установленных пакетов в системе. 
Из полученной таблицы создайте аналогичную в локальной базе данных sqlite3. Имя файла БД - ваша фамилия.
Из таблицы в БД получите список пакетов, имя которых начинается на букву r и содержит в номере версии слово ubuntu.
'''
import sys
import paramiko
import sqlite3
import pandas as pd
from pprint import pprint as pp

password = #и тут пароль

host = '192.168.56.101'
user = 'wiz'
port = 22

try:
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname=host, username=user, password=password, port=port)
    stdin , stdout, stderr = client.exec_command('dpkg --get-selections | grep -v deinstall')
    output = stdout.readlines()
    print("\n".join(output))
    con = sqlite3.connect('sobol.db')
    c = con.cursor()
#    c.execute('CREATE TABLE new_table (Name, Status)')
    for i in output:
        line = i.replace("install\n",'').replace("\t",'')
        sql = f"INSERT INTO new_table (Name) VALUES ('{line}')"
        c.execute(sql)
    con.commit()
    c.execute('select * from new_table')
    pp(c.fetchall())
finally:
    client.close()



accountsservice					install

acl						install

acpid						install

adduser						install

amd64-microcode					install

apparmor					install

apport						install

apport-symptoms					install

apt						install

apt-utils					install

at						install

base-files					install

base-passwd					install

bash						install

bash-completion					install

bc						install

bcache-tools					install

bind9-host					install

binutils					install

binutils-common:amd64				install

binutils-x86-64-linux-gnu			install

bsdmainutils					install

bsdutils					install

btrfs-progs					install

btrfs-tools					install

build-essential					install

busybox-initramfs				install

busybox-static					install

byobu						install

bzip2						install

ca-certificates					install

cloud-guest-utils				install

cloud-init					install

cloud-initramfs-copymods			install

cloud-initramfs-dyn-netconf			install

command-not-found				install

command-not-found-data				install

console-setup					install

console-setup-linux	

In [57]:
'''
Задача 3
Используя pandas, прочитайте CSV-данные (http://samplecsvs.s3.amazonaws.com/SalesJan2009.csv) в датафрейм. 
Определите среднее значение цены для всех платежей с Mastercard. Постройте графики цен для каждого типа платежного средства.
'''

import pandas as pd
import requests 

response = requests.get('http://samplecsvs.s3.amazonaws.com/SalesJan2009.csv')
csv_data = response.text
with open('data.csv', 'w') as output_file:
    output_file.write(csv_data)
df = pd.read_csv('data.csv')
df['Price'] = pd.to_numeric(df['Price'],errors='coerce')

In [69]:
df['Payment_Type'] = 'Mastercard'
df.groupby('Payment_Type')['Price'].mean()

Payment_Type
Mastercard    1622.367101
Name: Price, dtype: float64